In [28]:
import numpy as np
import pandas as pd
from scipy import linalg
from math import sqrt
from tabulate import tabulate



In [29]:
np.set_printoptions(formatter={'float_kind':"{:.8f}".format})
pd.options.display.float_format = '{:.8f}'.format

# Data creation

In [30]:
# def b(a):
#     size=a.shape[0]
#     return np.random.randint(100, size=(size))

In [31]:
def pretty_view(a):
    pret=pd.DataFrame(data=a, index=list(range(1, a.shape[0]+1)), columns=list(range(1, a.shape[1]+1)))
#     pret['b']=b
    display(pret)

In [32]:
h3=linalg.hilbert(3)
# b3=b(h3)
pretty_view(h3)

,1,2,3
1,1.00000000,0.50000000,0.33333333
2,0.50000000,0.33333333,0.25000000
3,0.33333333,0.25000000,0.20000000


In [33]:
h4=linalg.hilbert(4)
# b4=b(h4)
pretty_view(h4)

,1,2,3,4
1,1.00000000,0.50000000,0.33333333,0.25000000
2,0.50000000,0.33333333,0.25000000,0.20000000
3,0.33333333,0.25000000,0.20000000,0.16666667
4,0.25000000,0.20000000,0.16666667,0.14285714


In [34]:
h5=linalg.hilbert(5)
# b5=b(h5)
pretty_view(h5)

,1,2,3,4,5
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111


In [35]:
def count_b(L, x):  # считаем b для последующих вычислений
    b = np.dot(L, x) # a dot b = sum(ai*bi)
    return b

In [36]:
def sqrt_method(A):  # Находим L из А
    L = np.zeros((A.shape[0], A.shape[0]))
    for i in range(A.shape[0]):
        for j in range(i):
            temp = 0
            for k in range(j):
                temp += L[i][k] * L[j][k]
            L[i][j] = (A[i][j] - temp) / L[j][j]
        temp = A[i][i]
        for k in range(i):
            temp -= L[i][k] * L[i][k]
        L[i][i] = sqrt(temp)
    return L

In [37]:
def solve(L, b):  # решаем регуляризационную систему, находим x, y
    y = np.linalg.solve(L, b)
    x = np.linalg.solve(L.transpose(), y)
    return x

In [38]:
def variation_matrix(A, alpha, b, x, df):
    df.append([alpha, np.linalg.cond(A), np.linalg.cond(A + alpha * np.eye(A.shape[0])),
                 np.linalg.norm(x - solve(sqrt_method(A + alpha * np.eye(A.shape[0])), b))])
    return df


In [39]:
def print_all(A, df, alpha, diff_matr): # строим красивые таблички
    print("Матрица:")
    print(*A, sep='\n')
    print()
    print(tabulate(df, headers=['alpha', 'cond(A)', 'cond(A + alpha * E)', '||x - x_a||'],
                    tablefmt='fancy_grid', numalign="right"))
    print()
    print("Наилучшее значение alpha =", alpha)
    print()
    print("||x - x_a|| для различных матриц:")
    print(tabulate(diff_matr,
                   headers=['Ax = b', 'A + alpha * x = b', 'A + 10 * alpha * x = b', 'A + 0.1 * alpha * x = b'],
                   tablefmt='fancy_grid', numalign="right"))
    print()

In [40]:
def final_count(data):
    A = np.array(data, dtype=float)
    x = np.ones(A.shape[0])
    b = count_b(A, x)
    df=[]
    for i in (-2, -3, -5, -7, -9, -12):
        variation_matrix(A, 10 ** i, b, x, df)
    x = np.random.uniform(0, 100, size=A.shape[0])
    b = count_b(A, x)
    min_diff = df[0][3]
    for i in range(len(df)):
        if min_diff > df[i][3]:
            min_diff = df[i][3]
            alpha = df[i][0]
    diff_matr = [[np.linalg.norm(x - solve(sqrt_method(A), b)),
                  np.linalg.norm(x - solve(sqrt_method(A + alpha * np.eye(A.shape[0])), b)), # eye делает квадратную матрицу с единицами на диагонали
                  np.linalg.norm(x - solve(sqrt_method(A + 10 * alpha * np.eye(A.shape[0])), b)),
                  np.linalg.norm(x - solve(sqrt_method(A + 0.1 * alpha * np.eye(A.shape[0])), b))]]
    return print_all(A, df, alpha, diff_matr)


In [41]:
final_count(h3)

Матрица:
[1.00000000 0.50000000 0.33333333]
[0.50000000 0.33333333 0.25000000]
[0.33333333 0.25000000 0.20000000]

╒═════════╤═══════════╤═══════════════════════╤═══════════════╕
│   alpha │   cond(A) │   cond(A + alpha * E) │   ||x - x_a|| │
╞═════════╪═══════════╪═══════════════════════╪═══════════════╡
│    0.01 │   524.057 │                111.79 │     0.0945095 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   0.001 │   524.057 │               382.205 │     0.0283085 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-05 │   524.057 │               522.118 │   0.000383955 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-07 │   524.057 │               524.037 │   3.85343e-06 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-09 │   524.057 │               524.057 │   3.85357e-08 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-12 │   524.057 │               524.057 │   3.8

In [42]:
final_count(h4)

Матрица:
[1.00000000 0.50000000 0.33333333 0.25000000]
[0.50000000 0.33333333 0.25000000 0.20000000]
[0.33333333 0.25000000 0.20000000 0.16666667]
[0.25000000 0.20000000 0.16666667 0.14285714]

╒═════════╤═══════════╤═══════════════════════╤═══════════════╕
│   alpha │   cond(A) │   cond(A + alpha * E) │   ||x - x_a|| │
╞═════════╪═══════════╪═══════════════════════╪═══════════════╡
│    0.01 │   15513.7 │               149.575 │      0.117184 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   0.001 │   15513.7 │               1368.84 │     0.0310994 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-05 │   15513.7 │               14059.9 │    0.00214018 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-07 │   15513.7 │               15497.7 │    2.3559e-05 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-09 │   15513.7 │               15513.6 │   2.35828e-07 │
├─────────┼───────────┼───────────────

In [43]:
final_count(h5)

Матрица:
[1.00000000 0.50000000 0.33333333 0.25000000 0.20000000]
[0.50000000 0.33333333 0.25000000 0.20000000 0.16666667]
[0.33333333 0.25000000 0.20000000 0.16666667 0.14285714]
[0.25000000 0.20000000 0.16666667 0.14285714 0.12500000]
[0.20000000 0.16666667 0.14285714 0.12500000 0.11111111]

╒═════════╤═══════════╤═══════════════════════╤═══════════════╕
│   alpha │   cond(A) │   cond(A + alpha * E) │   ||x - x_a|| │
╞═════════╪═══════════╪═══════════════════════╪═══════════════╡
│    0.01 │    476607 │               157.653 │      0.130061 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   0.001 │    476607 │               1562.91 │     0.0387793 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-05 │    476607 │                117931 │    0.00379261 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│   1e-07 │    476607 │                462539 │   0.000139421 │
├─────────┼───────────┼───────────────────────┼───────────────┤
│